In [2]:
import cv2
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_resnet_features(image):
  
    img = cv2.resize(image, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)

    features = resnet_model.predict(img)
    return features

sample = cv2.imread("SOCOFing/Altered/Altered-Hard/150__M_Right_index_finger_Obl.BMP")

best_score = 0 
filename = None
image = None
best_features = None

counter = 0
for file in os.listdir("SOCOFing/Real")[:1000]:
    fingerprint_image = cv2.imread(os.path.join("SOCOFing/Real", file))


    fingerprint_features = extract_resnet_features(fingerprint_image)
    sample_features = extract_resnet_features(sample)

    similarity_score = np.dot(sample_features, fingerprint_features.T) / (np.linalg.norm(sample_features) * np.linalg.norm(fingerprint_features))

    if similarity_score > best_score:
        best_score = similarity_score
        filename = file
        image = fingerprint_image
        best_features = fingerprint_features

        print("BEST MATCH:", filename) 
        print("SCORE:", best_score * 100, "%")

cv2.imshow("Best Match", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 285ms/step
BEST MATCH: 100__M_Left_index_finger.BMP
SCORE: [[76.00472]] %
1/1 [==============================] - 0s 263ms/step
BEST MATCH: 100__M_Left_little_finger.BMP
SCORE: [[84.33277]] %
1/1 [==============================] - 0s 287ms/step
BEST MATCH: 100__M_Right_little_finger.BMP
SCORE: [[84.77103]] %
1/1 [==============================] - 0s 318ms/step
BEST MATCH: 101__M_Left_index_finger.BMP
SCORE: [[84.84886]] %
1/1 [==============================] - 0s 283ms/step


KeyboardInterrupt: 

In [5]:
import cv2
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc


def load_data(data_dir):
    """Loads fingerprint images and labels from a directory."""
    X, y = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_height, img_width))
            img = preprocess_input(img)
            X.append(img)
            y.append(label)  # Assuming label is the directory name
    return np.array(X), np.array(y)


def extract_features(img):
    """Extracts features from a fingerprint image using pre-trained ResNet50."""
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    img = np.expand_dims(img, axis=0)
    features = resnet_model.predict(img)
    return features


def create_model(learning_rate=0.001):
    """Creates a classification model on top of pre-trained ResNet50 features."""
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    for layer in resnet_model.layers:
        layer.trainable = False  # Freeze pre-trained layers

    x = resnet_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)  # Add a dense layer with ReLU activation
    output = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation

    model = Model(inputs=resnet_model.input, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model


def train_model(model, X_train, y_train, X_val, y_val, epochs=10):
    """Trains the model on the training data and evaluates on the validation data."""
    datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    datagen.fit(X_train)

    model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))
    return model


def evaluate_model(model, X_test, y_test):
    """Evaluates the model on the test data and calculates EER and ROC AUC."""
    y_pred = model.predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)

    # Calculate EER (Equal Error Rate)
    eer_threshold = thresholds[np.argmin(np.abs(fpr - tpr))]
    eer = (fpr[np.argmin(np.abs(fpr - tpr))] + tpr[np.argmin(np.abs(fpr - tpr))]) / 2

    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"EER: {eer:.4f}")


# Define paths to fingerprint images
data_dir = "SOCOFing"  # Replace with your data directory path
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")  # Add a test directory for evaluation
